# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

In [51]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

You should consider upgrading via the 'C:\Users\brics\Documents\git_repos\git_repos\lang-chain\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [52]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [53]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.environ['OPENAI_MODEL_NAME'] = os.getenv('OPENAI_API_KEY')
os.environ['SERPER_API_KEY'] = os.getenv('SERPER_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [54]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [55]:
from crewai import Agent, Task, Crew

## Creating Agents

In [56]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [57]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [58]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [59]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()
DirectoryReadTool(directory='./instructions')

DirectoryReadTool(name='List files in directory', description="List files in directory() - A tool that can be used to list ./instructions's content.", args_schema=<class 'crewai_tools.tools.directory_read_tool.directory_read_tool.FixedDirectoryReadToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x00000238A069D360>, directory='./instructions')

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [60]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [61]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [62]:
sentiment_analysis_tool = SentimentAnalysisTool() # Initialize the tool instance 

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [63]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [64]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [65]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

2024-05-17 22:48:43,490 - 16664 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [66]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a detailed profile for our engagement strategy. I should focus on key decision-makers, recent business developments, and potential needs.

Action: Read a file's content
Action Input: {"file_path": "DeepLearningAI/company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No su

- Display the final result as Markdown.

In [67]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng:

Subject: Empowering AI Education with Innovative Solutions

Dear Andrew Ng,

I hope this message finds you well. I am reaching out to express my excitement about DeepLearningAI's recent product launch and the incredible strides your team has made in advancing AI education globally.

As a global leader in AI, your dedication to making world-class AI education accessible to individuals worldwide is truly inspiring. The Batch AI News and Insights, particularly the use of large language models for autonomous decision-making, showcases DeepLearningAI's commitment to pushing the boundaries of AI education.

At CrewAI, we share your passion for democratizing AI education and believe that our innovative learning solutions can complement DeepLearningAI's mission. Our expert-led educational experiences and hands-on training platforms are designed to enhance collaborative learning experiences and provide users with the tools they need to succeed in the field of AI.

I am confident that our personalized engagement strategies and cutting-edge tools can support DeepLearningAI in furthering its goal of empowering individuals through AI education. I would love the opportunity to discuss how CrewAI can tailor our solutions to meet the unique needs of DeepLearningAI and contribute to your ongoing success.

Thank you for your time, Andrew. I look forward to the possibility of collaborating with DeepLearningAI and supporting your vision for the future of AI education.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Please tailor the sender's name according to your actual name and position, and ensure the email draft resonates with DeepLearningAI's values and recent achievements.